In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

# Experimental result  vs JRC , permanent, 2000-2020

In [ ]:
pw = []
jrc = []
for year in range(2000,2021,1):
    pw_area = ee.Image('users/311605001111/WF/tibet_WF_' + str(year)).gte(0.75).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    pw.append(areas.get('frequency').getInfo())
    JRC = ee.Image('JRC/GSW1_3/YearlyHistory/' + str(year)).gte(3).multiply(ee.Image.pixelArea()).divide(1e6)
    jrc_areas = JRC.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    jrc.append(jrc_areas.get('waterClass').getInfo())
print(pw)
print(jrc)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# plot permanent water and seasonal water(折线图)
plt.figure(figsize=(8,6))
year = np.arange(2000,2021,1)
plt.plot(year, jrc,'*-b' ,label = 'JRC')
plt.plot(year, pw,'^--r',label = 'Result')
plt.title("Annual minimum water area(region C)", fontsize=20)
plt.xlabel("Year", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)


plt.tick_params(labelsize=15)
plt.xticks([2000,2005,2010,2015,2020])
plt.legend(prop={'size':16})
plt.show()

# 两个时期的变化对比

In [4]:
def maskSR(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)
l2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate('2013-01-01', '2020-12-31')
l2000 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate('2000-01-01', '2012-12-31')
Map.addLayer(l2020.map(maskSR).median().clip(roi), {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4},'image 2020')
Map.addLayer(l2000.map(maskSR).median().clip(roi), {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4},'image 2000')


wf0012 = ee.Image("users/311605001111/WF/wuhan_WF_9912").gte(0.75).multiply(-2)
Map.addLayer(wf0012,{'palette':['white','blue'],'min':0.25,'max':1},"0012")
wf1320 = ee.Image("users/311605001111/WF/wuhan_WF_1320").gte(0.75)
Map.addLayer(wf1320,{'palette':['white','blue'],'min':0.25,'max':1},"1320")
change = wf1320.subtract(wf0012)
Map.addLayer(change,{'palette':['white','purple','red','blue'],'min':0,'max':3},"change")
# Map.addLayer(change.selfMask(),{'palette':['blue']},"change")

In [6]:
change_area = change.eq(1).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(change_area.get('frequency').getInfo())
total_area = change.gte(1).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(total_area.get('frequency').getInfo())

print(change_area.get('frequency').getInfo()/total_area.get('frequency').getInfo())

242.01061075082345
1961.4510441465134
0.12338345709572862


In [ ]:
#### wuhan,放大区域
## a
114.4418, 30.7134,114.5024, 30.7618
## b
114.0303, 30.6364,114.0838, 30.6842
## c
114.3783, 30.5665,114.4162, 30.5944
## d
113.7921, 30.2900,113.8624, 30.3382
## e
113.7753, 30.1524,113.8238, 30.1999

#### tibet,放大区域
## a
90.5181, 35.1563,90.6495, 35.2483
## b
90.2220, 34.7622,90.2968, 34.8177

In [ ]:
# 水体频率
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
year = 2020
region = [90.2220, 34.7622,90.2968, 34.8177] 
fig = plt.figure(figsize=(12, 8))
vis = {'palette':['white','purple','red','blue'],'min':0,'max':3}
pw = change.visualize(**vis)
Map.addLayer(pw,{},'pw')
ax = cartoee.get_map(pw,region=region)
# cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
# cartoee.add_colorbar(ax, vis,cmap = 'Blues',loc="right",label="water frequency", orientation="vertical")
# ax.set_title(label = 'waterbody frequency in 2020(landsat7)', fontsize=20)

# 各类水体占比

In [ ]:
# 各类水体占比，wuhan，2020
wf = ee.Image("users/311605001111/WF/wuhan_WF_2020")
# Map.addLayer(wf,{'palette':['white','green'],'min':0,'max':1},"water frequency")

basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
wf_permanent = wf.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
wf_season= ee.ImageCollection([wf.gte(0.25),wf.lt(0.75)]).sum().eq(2).rename('waterclass')
wf_waterclass = ee.ImageCollection([wf_permanent,wf_season,basemap]).sum()

pw_area = wf_waterclass.gte(1).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(pw_area.get('waterclass').getInfo())
area = wf_waterclass.gte(0).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(area.get('waterclass').getInfo())
print(pw_area.get('waterclass').getInfo()/area.get('waterclass').getInfo())


# 各类水体占比，wuhan，2020
wf = ee.Image("users/311605001111/WF/tibetan_WF_2020")
# Map.addLayer(wf,{'palette':['white','green'],'min':0,'max':1},"water frequency")

basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
wf_permanent = wf.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
wf_season= ee.ImageCollection([wf.gte(0.25),wf.lt(0.75)]).sum().eq(2).rename('waterclass')
wf_waterclass = ee.ImageCollection([wf_permanent,wf_season,basemap]).sum()

pw_area = wf_waterclass.gte(1).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(pw_area.get('waterclass').getInfo())
area = wf_waterclass.gte(0).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(area.get('waterclass').getInfo())
print(pw_area.get('waterclass').getInfo()/area.get('waterclass').getInfo())

In [ ]:
# 各类水体面积制图，直方图
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.title("region E", fontsize=20)
plt.bar([1, 3, 5], [2572.576, 85.356, 14827.301], label='2020')
plt.bar([2, 4, 6], [2107.397, 316.785, 15061.051], label='1999-2020')
plt.xlabel("water type", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)
plt.tick_params(labelsize=15)
plt.xticks([1.5,3.5,5.5],['PW','SW','NW'])
plt.legend(prop={'size':18})
plt.show()

# 各类水体面积制图，直方图
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.title("region C", fontsize=20)
plt.bar([1, 3, 5], [1967.926, 1135.848, 12254.523], label='2020')
plt.bar([2, 4, 6], [1685.267, 1447.814, 12225.217], label='1999-2020')
plt.xlabel("water type", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)
plt.tick_params(labelsize=15)
plt.xticks([1.5,3.5,5.5],['PW','SW','NW'])
plt.legend(prop={'size':18})
plt.show()

# 比较，2020，1999-2020

In [ ]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# tibet
region = 'wuhan'

wf = ee.Image('users/311605001111/WF/' + region + '_WF_2020')
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
wf_permanent = wf.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
wf_season= ee.ImageCollection([wf.gte(0.25),wf.lt(0.75)]).sum().eq(2).rename('waterclass')
wf_waterclass = ee.ImageCollection([wf_permanent,wf_season,basemap]).sum()
pw_area = wf_waterclass.eq(1).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(pw_area.get('waterclass').getInfo())
area = wf_waterclass.gte(1).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(area.get('waterclass').getInfo())
# print(pw_area.get('waterclass').getInfo()/ 15374.677415453567)
print(pw_area.get('waterclass').getInfo()/ area.get('waterclass').getInfo())

wf = ee.Image('users/311605001111/WF/' + region + '_WF_9920')
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
wf_permanent = wf.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
wf_season= ee.ImageCollection([wf.gte(0.25),wf.lt(0.75)]).sum().eq(2).rename('waterclass')
wf_waterclass = ee.ImageCollection([wf_permanent,wf_season,basemap]).sum()
print('pw')
pw_area = wf_waterclass.eq(1).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(pw_area.get('waterclass').getInfo())
area = wf_waterclass.gte(1).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(area.get('waterclass').getInfo())
# print(pw_area.get('waterclass').getInfo()/ 15374.677415453567)
print(pw_area.get('waterclass').getInfo()/ area.get('waterclass').getInfo())

In [ ]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# tibet
region = 'tibet'

wf = ee.Image('users/311605001111/WF/' + region + '_WF_2020')
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
wf_permanent = wf.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
wf_season= ee.ImageCollection([wf.gte(0.25),wf.lt(0.75)]).sum().eq(2).rename('waterclass')
wf_waterclass = ee.ImageCollection([wf_permanent,wf_season,basemap]).sum()
pw_area = wf_waterclass.eq(0).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(pw_area.get('waterclass').getInfo())
# area = wf_waterclass.gte(1).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
# print(area.get('waterclass').getInfo())
print(pw_area.get('waterclass').getInfo()/ 17485.59784367786)
# print(pw_area.get('waterclass').getInfo()/ area.get('waterclass').getInfo())


wf = ee.Image('users/311605001111/WF/' + region + '_WF_9920')
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
wf_permanent = wf.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
wf_season= ee.ImageCollection([wf.gte(0.25),wf.lt(0.75)]).sum().eq(2).rename('waterclass')
wf_waterclass = ee.ImageCollection([wf_permanent,wf_season,basemap]).sum()
print('pw')
pw_area = wf_waterclass.eq(0).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print(pw_area.get('waterclass').getInfo())
# area = wf_waterclass.gte(1).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
# print(area.get('waterclass').getInfo())
print(pw_area.get('waterclass').getInfo()/ 17485.59784367786)
# print(pw_area.get('waterclass').getInfo()/ area.get('waterclass').getInfo())

# 计算水体频率，2000-2012，2013-2020，2000-2020

##  tibetan

In [ ]:
# 有效像元
image99 = ee.Image("users/311605001111/WP-VP/tibet_VP_1999")
image0 = ee.Image("users/311605001111/WP-VP/tibet_VP_2000")
image1 = ee.Image("users/311605001111/WP-VP/tibet_VP_2001")
image2 = ee.Image("users/311605001111/WP-VP/tibet_VP_2002")
image3 = ee.Image("users/311605001111/WP-VP/tibet_VP_2003")
image4 = ee.Image("users/311605001111/WP-VP/tibet_VP_2004")
image5 = ee.Image("users/311605001111/WP-VP/tibet_VP_2005")
image6 = ee.Image("users/311605001111/WP-VP/tibet_VP_2006")
image7 = ee.Image("users/311605001111/WP-VP/tibet_VP_2007")
image8 = ee.Image("users/311605001111/WP-VP/tibet_VP_2008")
image9 = ee.Image("users/311605001111/WP-VP/tibet_VP_2009")
image10 = ee.Image("users/311605001111/WP-VP/tibet_VP_2010")
image11 = ee.Image("users/311605001111/WP-VP/tibet_VP_2011")
image12 = ee.Image("users/311605001111/WP-VP/tibet_VP_2012")
image13 = ee.Image("users/311605001111/WP-VP/tibet_VP_2013")
image14 = ee.Image("users/311605001111/WP-VP/tibet_VP_2014")
image15 = ee.Image("users/311605001111/WP-VP/tibet_VP_2015")
image16 = ee.Image("users/311605001111/WP-VP/tibet_VP_2016")
image17 = ee.Image("users/311605001111/WP-VP/tibet_VP_2017")
image18 = ee.Image("users/311605001111/WP-VP/tibet_VP_2018")
image19 = ee.Image("users/311605001111/WP-VP/tibet_VP_2019")
image20 = ee.Image("users/311605001111/WP-VP/tibet_VP_2020")

validPixel = ee.ImageCollection([image99,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,image10,image11,image12,image13,image14,image15,image16,image17,image18,image19,image20]).sum()
validPixel13 = ee.ImageCollection([image99,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,image10,image11,image12]).sum()
validPixel8 = ee.ImageCollection([image13,image14,image15,image16,image17,image18,image19,image20]).sum()

In [ ]:
# 水体像元
image99 = ee.Image("users/311605001111/WP-VP/tibet_WP_1999")
image0 = ee.Image("users/311605001111/WP-VP/tibet_WP_2000")
image1 = ee.Image("users/311605001111/WP-VP/tibet_WP_2001")
image2 = ee.Image("users/311605001111/WP-VP/tibet_WP_2002")
image3 = ee.Image("users/311605001111/WP-VP/tibet_WP_2003")
image4 = ee.Image("users/311605001111/WP-VP/tibet_WP_2004")
image5 = ee.Image("users/311605001111/WP-VP/tibet_WP_2005")
image6 = ee.Image("users/311605001111/WP-VP/tibet_WP_2006")
image7 = ee.Image("users/311605001111/WP-VP/tibet_WP_2007")
image8 = ee.Image("users/311605001111/WP-VP/tibet_WP_2008")
image9 = ee.Image("users/311605001111/WP-VP/tibet_WP_2009")
image10 = ee.Image("users/311605001111/WP-VP/tibet_WP_2010")
image11 = ee.Image("users/311605001111/WP-VP/tibet_WP_2011")
image12 = ee.Image("users/311605001111/WP-VP/tibet_WP_2012")
image13 = ee.Image("users/311605001111/WP-VP/tibet_WP_2013")
image14 = ee.Image("users/311605001111/WP-VP/tibet_WP_2014")
image15 = ee.Image("users/311605001111/WP-VP/tibet_WP_2015")
image16 = ee.Image("users/311605001111/WP-VP/tibet_WP_2016")
image17 = ee.Image("users/311605001111/WP-VP/tibet_WP_2017")
image18 = ee.Image("users/311605001111/WP-VP/tibet_WP_2018")
image19 = ee.Image("users/311605001111/WP-VP/tibet_WP_2019")
image20 = ee.Image("users/311605001111/WP-VP/tibet_WP_2020")

waterPixel = ee.ImageCollection([image99,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,image10,image11,image12,image13,image14,image15,image16,image17,image18,image19,image20]).sum()
waterPixel13 = ee.ImageCollection([image99,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,image10,image11,image12]).sum()
waterPixel8 = ee.ImageCollection([image13,image14,image15,image16,image17,image18,image19,image20]).sum()

In [ ]:
waterfrequency = waterPixel.divide(validPixel).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")

dataset_id = 'tibet_WF_' + '9920'
assetID = 'users/311605001111/WF/' + dataset_id
task = ee.batch.Export.image.toAsset(**{
    'image': waterfrequency,
    'description': dataset_id,
    'assetId': assetID,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

In [ ]:
waterfrequency13 = waterPixel13.divide(validPixel13).rename('frequency')
# Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")

dataset_id = 'tibet_WF_' + '9912'
assetID = 'users/311605001111/WF/' + dataset_id
task = ee.batch.Export.image.toAsset(**{
    'image': waterfrequency13,
    'description': dataset_id,
    'assetId': assetID,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

In [ ]:
waterfrequency8 = waterPixel8.divide(validPixel8).rename('frequency')
# Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")
dataset_id = 'tibet_WF_' + '1320'
assetID = 'users/311605001111/WF/' + dataset_id
task = ee.batch.Export.image.toAsset(**{
    'image': waterfrequency8,
    'description': dataset_id,
    'assetId': assetID,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

## wuhan

In [ ]:
# 有效像元
image99 = ee.Image("users/311605001111/WP-VP/wuhan_VP_1999").float()
image0 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2000").float()
image1 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2001").float()
image2 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2002").float()
image3 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2003").float()
image4 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2004").float()
image5 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2005").float()
image6 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2006").float()
image7 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2007").float()
image8 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2008").float()
image9 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2009").float()
image10 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2010").float()
image11 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2011").float()
image12 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2012").float()
image13 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2013").float()
image14 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2014").float()
image15 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2015").float()
image16 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2016").float()
image17 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2017").float()
image18 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2018").float()
image19 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2019").float()
image20 = ee.Image("users/311605001111/WP-VP/wuhan_VP_2020").float()

validPixel = ee.ImageCollection([image99,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,image10,image11,image12,image13,image14,image15,image16,image17,image18,image19,image20]).sum()
validPixel13 = ee.ImageCollection([image99,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,image10,image11,image12]).sum()
validPixel8 = ee.ImageCollection([image13,image14,image15,image16,image17,image18,image19,image20]).sum()

In [ ]:
# 水体像元
image99 = ee.Image("users/311605001111/WP-VP/wuhan_WP_1999").float()
image0 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2000").float()
image1 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2001").float()
image2 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2002").float()
image3 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2003").float()
image4 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2004").float()
image5 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2005").float()
image6 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2006").float()
image7 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2007").float()
image8 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2008").float()
image9 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2009").float()
image10 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2010").float()
image11 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2011").float()
image12 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2012").float()
image13 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2013").float()
image14 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2014").float()
image15 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2015").float()
image16 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2016").float()
image17 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2017").float()
image18 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2018").float()
image19 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2019").float()
image20 = ee.Image("users/311605001111/WP-VP/wuhan_WP_2020").float()

waterPixel = ee.ImageCollection([image99,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,image10,image11,image12,image13,image14,image15,image16,image17,image18,image19,image20]).sum()
waterPixel13 = ee.ImageCollection([image99,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,image10,image11,image12]).sum()
waterPixel8 = ee.ImageCollection([image13,image14,image15,image16,image17,image18,image19,image20]).sum()

In [ ]:
waterfrequency = waterPixel.divide(validPixel).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")

dataset_id = 'wuhan_WF_' + '9920'
assetID = 'users/311605001111/WF/' + dataset_id
task = ee.batch.Export.image.toAsset(**{
    'image': waterfrequency,
    'description': dataset_id,
    'assetId': assetID,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

In [ ]:
waterfrequency13 = waterPixel13.divide(validPixel13).rename('frequency')
# Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")

dataset_id = 'wuhan_WF_' + '9912'
assetID = 'users/311605001111/WF/' + dataset_id
task = ee.batch.Export.image.toAsset(**{
    'image': waterfrequency13,
    'description': dataset_id,
    'assetId': assetID,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

In [ ]:
waterfrequency8 = waterPixel8.divide(validPixel8).rename('frequency')
# Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")

dataset_id = 'wuhan_WF_' + '1320'
assetID = 'users/311605001111/WF/' + dataset_id
task = ee.batch.Export.image.toAsset(**{
    'image': waterfrequency8,
    'description': dataset_id,
    'assetId': assetID,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

# 计算面积及趋势，校正前与校正后

## 最小面积

In [ ]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# 计算面积
re = 'wuhan'
pw_no = []
pw = []
for year in range(2000,2021,1):
    pw_area = ee.Image('users/311605001111/WF/'+ re + '_WF_' + str(year)).gte(0.75).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    pw.append(areas.get('frequency').getInfo())
    pw_no_area = ee.Image('users/311605001111/WF_nocorrect/' + re + '_WF_' + str(year)).gte(0.75).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_no_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    pw_no.append(areas.get('frequency').getInfo())

print(pw)
print(pw_no)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(8,6))
year = np.arange(2000,2021,1)
plt.plot(year, pw_no,'*-b' ,label = 'Before correction')
plt.plot(year, pw,'^--r',label = 'After correction')
plt.title("Annual minimum water area(region C)", fontsize=20)
plt.xlabel("Year", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)

year = np.arange(2000,2021,1).reshape(-1,1)
model = LinearRegression()  # 创建模型
model.fit(year, pw)  # 拟合
x = [[2000],[2005],[2010],[2015],[2020]]
y = model.predict(x)
plt.plot(x, y, 'g-',label = 'Fitting trend')

plt.tick_params(labelsize=15)
plt.xticks([2000,2005,2010,2015,2020])
plt.legend(prop={'size':15})
plt.show()

In [ ]:
## WUHAN
wuhan = [2398.950777750184, 1742.8030615226294, 1828.2091128396241, 2090.4535433473698, 2138.736503590754, 1739.0688696166856, 1767.3411693673304, 1710.2390605628846, 1815.8135340201165, 1706.0139576459253, 1882.1369948587621, 2014.3360468078395, 1833.225413689255, 2055.943986936598, 1889.5416669547126, 1870.0400291467226, 1881.0473253924222, 1942.578887656528, 1862.986985017366, 1842.0526327837024, 1872.7606165067114, 1967.9265366016166]
wuhan_no = [2457.136192107628, 1674.0707124845433, 1752.8559947485119, 2010.7967729997138, 2071.19157907065, 1811.5829773712348, 1617.0989891192637, 1703.7302437256399, 1685.3743135839816, 1728.6444598924727, 1845.7745514544379, 2045.2888031372784, 1734.4539147723572, 1958.5449877728195, 1873.5955681731118, 1871.3600703869186, 1930.062232861054, 1979.1979729923567, 1827.5594563310176, 1794.7825691585028, 1833.0607390354073, 1953.5220370067966]

In [ ]:
## tibet
tibet = [1782.2733800481428, 1795.614042410124, 1762.8202364992812, 1822.7315809449021, 1958.1379549389019, 1986.9382663011393, 2032.8778920316033, 2067.4037299380448, 2072.462038737297, 2086.9691817955936, 2116.9879140745456, 2159.4533750086784, 2240.115411350907, 2276.329986691964, 2322.7076210290734, 2268.1952218014376, 2287.5011316323553, 2260.6199980205656, 2315.5543970223653, 2372.009359575668, 2599.3791057861467, 2572.5756991874923]
tibet_no = [1765.28922724515, 1793.3787730584177, 1767.024804868841, 1828.767532018362, 1963.389024426214, 1974.1213245577803, 1993.023937641452, 2019.3218589689595, 2069.5799653607132, 2082.5413293942597, 2109.895312146683, 2177.649359272074, 2260.0860751694813, 2265.7539541586307, 2338.1338218471406, 2312.0242609540533, 2331.585749846577, 2303.2956855327966, 2358.61739627326, 2369.998679345598, 2527.9263220197727, 2574.342391361047]

In [ ]:
# 计算变化率
q = wuhan
rt = []
ch = []
for i in range(1,22,1):
    rate = ((q[i] - q[i-1])/ q[i-1])*100
    rt.append(rate)
    change = q[i] - q[i-1]
    ch.append(change)

print(rt)
print(ch)

print('min')
print(rt.index(min(rt)) + 2000)
print(min(rt))
print(ch[rt.index(min(rt))])

print('max')
print(rt.index(max(rt)) + 2000)
print(max(rt))
print(ch[rt.index(max(rt))])

In [ ]:
q = wuhan
area = []
for i in range(0,22,1):
    rate = (q[i]/ 15374.677415453567) * 100
    area.append(rate)

print(q)
print(area)

print('min')
print(area.index(min(area)) + 1999)
print(min(area))
print(q[area.index(min(area))])

print('max')
print(area.index(max(area)) + 1999)
print(max(area))
print(q[area.index(max(area))])

## 最大面积

In [ ]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# 计算面积
re = 'wuhan'
pw_no = []
pw = []
for year in range(2000,2021,1):
    pw_area = ee.Image('users/311605001111/WF/'+ re + '_WF_' + str(year)).gte(0.25).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    pw.append(areas.get('frequency').getInfo())
    pw_no_area = ee.Image('users/311605001111/WF_nocorrect/' + re + '_WF_' + str(year)).gte(0.25).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_no_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    pw_no.append(areas.get('frequency').getInfo())

print(pw)
print(pw_no)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(8,6))
year = np.arange(2000,2021,1)
plt.plot(year, pw_no,'*-b' ,label = 'Before correction')
plt.plot(year, pw,'^--r',label = 'After correction')
plt.title("Annual maximum water area(region C)", fontsize=20)
plt.xlabel("Year", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)

year = np.arange(2000,2021,1).reshape(-1,1)
model = LinearRegression()  # 创建模型
model.fit(year, pw)  # 拟合
x = [[2000],[2005],[2010],[2015],[2020]]
y = model.predict(x)
plt.plot(x, y, 'g-',label = 'Fitting trend')

plt.tick_params(labelsize=15)
plt.xticks([2000,2005,2010,2015,2020])
plt.legend(prop={'size':15})
plt.show()

In [ ]:
## wuhan
wuhan = [4025.3122052125505, 2943.102517047763, 2922.9489180227474, 3354.159406783633, 3427.536155619044, 3010.801189581255, 3032.0549491848324, 2755.7707815095673, 2977.761331499893, 2935.8043150977874, 2954.909659473998, 3185.965902603649, 3021.295640554084, 3168.5326471552567, 2783.191822046776, 2879.8157243784813, 2841.867426644913, 2939.1305525922685, 2895.014091059462, 2922.4190100785745, 2881.678214724469, 3103.77478189314]
wuhan_no = [3878.958202332838, 2818.2595219398677, 2813.7504869118657, 3132.797263890634, 3322.918153084958, 2926.643229196115, 2701.629035183967, 2627.6673316895767, 2779.3520767819923, 2902.455644784429, 2825.0528482139307, 3120.2480117503396, 2695.8147808230506, 2978.547823133063, 2587.872288431195, 2747.4332223672823, 2568.5396323542536, 2996.9207133796813, 2759.5166260444526, 2590.7408817185224, 2639.6148390735043, 3057.659995232]

In [ ]:
## tibet
tibet = [1873.5692704849382, 1881.9383760012586, 1920.5577944327529, 2042.375092361062, 2093.4125657781856, 2088.7158678282667, 2145.9130526218664, 2152.2085717180958, 2169.3869019393055, 2177.0899761375053, 2229.6202141706713, 2288.0732105631987, 2396.616952137828, 2421.7569595520176, 2432.0276577346312, 2426.5877014260905, 2426.021269800562, 2427.4678722246103, 2494.1936548128374, 2641.9567676427973, 2678.8707099750045, 2657.931340974722]
tibet_no = [1837.7257387350464, 1877.5227279576743, 1913.7128994417287, 2023.9521999517074, 2063.296854045944, 2061.185020539416, 2111.1565627381055, 2121.2104277787707, 2152.6514793039655, 2152.5295163220985, 2213.9858939490273, 2284.8690345208242, 2359.4526954647076, 2403.519331716726, 2397.878161209701, 2393.3447181958486, 2394.4004138034, 2391.3992569219117, 2467.5488220585016, 2518.3605248060103, 2623.382365427767, 2643.9312035434777]

In [ ]:
# 计算变化率
q = wuhan
rt = []
ch = []
for i in range(1,22,1):
    rate = ((q[i] - q[i-1])/ q[i-1])*100
    rt.append(rate)
    change = q[i] - q[i-1]
    ch.append(change)

print(rt)
print(ch)

print('min')
print(rt.index(min(rt)) + 2000)
print(min(rt))
print(ch[rt.index(min(rt))])

print('max')
print(rt.index(max(rt)) + 2000)
print(max(rt))
print(ch[rt.index(max(rt))])

In [ ]:
q = wuhan
area = []
for i in range(0,22,1):
    rate = (q[i]/ 15374.677415453567) * 100
    area.append(rate)

print(q)
print(area)

print('min')
print(area.index(min(area)) + 1999)
print(min(area))
print(q[area.index(min(area))])

print('max')
print(area.index(max(area)) + 1999)
print(max(area))
print(q[area.index(max(area))])